# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Lab 2 - Parte 2</font>
## <font color='blue'>Regressão Logística x Torchvision Para Reconhecimento de Imagens</font>

Obs: Este Jupyter Notebook DEVE ser executado no ambiente em nuvem, como demonstrado nas aulas a seguir. 

## Instalando e Carregando os Pacotes

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
!pip install -q torch==1.13.0

In [ ]:
!pip install -q pytorch-lightning==1.8.3

In [ ]:
!pip install -q lightning-bolts

In [ ]:
# Imports
import math
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import gc
import types
import pkg_resources
import pytorch_lightning as pl

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

## Verificando o Ambiente de Desenvolvimento

In [ ]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print(f'\n{lable_3:-^100}')

In [ ]:
import platform
print(platform.platform())

In [ ]:
# Define o device (GPU ou CPU)
device = torch.device(processing_device)
print(device)

In [ ]:
# Imports
import os
import pandas as pd
import seaborn as sn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from IPython import display
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.classification import Accuracy

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

In [ ]:
# Seed para inicializar o processo randômico com o mesmo padrão
seed_everything(7)

In [ ]:
# Pasta para gravar os datasets
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")

In [ ]:
# Definimos o tamanho do batch de imagens de acordo com o hardware
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

In [ ]:
# Número de workers
NUM_WORKERS = int(os.cpu_count() / 2)

In [ ]:
# Módulo de transformações nos dados de treino (Data Loader de Treino)
prep_dados_treino = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding = 4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

In [ ]:
# Módulo de transformações nos dados de teste (teste após o treinamento) e validação (teste durante o treinamento)
prep_dados_teste = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

In [ ]:
# Módulo para carregar os dados e aplicar os data loaders
carrega_dados = CIFAR10DataModule(data_dir = PATH_DATASETS,
                                  batch_size = BATCH_SIZE,
                                  num_workers = NUM_WORKERS,
                                  train_transforms = prep_dados_treino,
                                  test_transforms = prep_dados_teste,
                                  val_transforms = prep_dados_teste)

In [ ]:
# Módulo para carregar um modelo pré-treinado de arquitetura ResNet sem os pesos (queremos somente a arquitetura)
def carrega_modelo_pretreinado():
    modelo = torchvision.models.resnet18(weights = None, num_classes = 10)
    modelo.conv1 = nn.Conv2d(3, 64, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1), bias = False)
    modelo.maxpool = nn.Identity()
    return modelo

In [ ]:
# Classe com Arquitetura do Modelo
class ModeloResnet(LightningModule):
    
    # Método construtor
    def __init__(self, lr = 0.05):
        super().__init__()
        self.save_hyperparameters()
        self.model = carrega_modelo_pretreinado()

    # Método Forward
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim = 1)

    # Método de um passo de treinamento
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    # Método de avaliação
    def evaluate(self, batch, stage = None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim = 1)
        accuracy = Accuracy(task = "multiclass", num_classes = 10).to(device)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar = True)
            self.log(f"{stage}_acc", acc, prog_bar = True)

    # Método de um passo de validação
    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    # Método de um passo de teste
    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    # Método de configuração do otimizador
    def configure_optimizers(self):
        
        # Otimização SGD
        optimizer = torch.optim.SGD(self.parameters(), 
                                    lr = self.hparams.lr, 
                                    momentum = 0.9, 
                                    weight_decay = 5e-4)
        
        # Passos por época
        steps_per_epoch = 45000 // BATCH_SIZE
        
        # Scheduler
        scheduler_dict = {
            "scheduler": OneCycleLR(optimizer,
                                    0.1,
                                    epochs = self.trainer.max_epochs,
                                    steps_per_epoch = steps_per_epoch),
            "interval": "step",
        }
        
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

In [ ]:
# Cria o modelo (Objeto = Instância da Classe)
modelo_dl = ModeloResnet(lr = 0.05)

In [ ]:
# Módulo de treinamento
treinador = Trainer(max_epochs = 30,
                    accelerator = "auto",
                    devices = 1 if torch.cuda.is_available() else None,  
                    logger = CSVLogger(save_dir = "logs/"),
                    callbacks = [LearningRateMonitor(logging_interval = "step"), 
                                 TQDMProgressBar(refresh_rate = 10)],
)

In [ ]:
# Treinamento
treinador.fit(modelo_dl, carrega_dados)

In [ ]:
# Avaliação do Modelo
treinador.test(modelo_dl, datamodule = carrega_dados)

In [ ]:
# Conseguimos performance bem superior ao modelo com Regressão Logística.

In [ ]:
# Carrega as métricas
metricas = pd.read_csv(f"{treinador.logger.log_dir}/metrics.csv")

In [ ]:
# Deleta os passos individuais
del metricas["step"]

In [ ]:
# Ajusta o índice
metricas.set_index("epoch", inplace = True)

In [ ]:
# Resultado
sn.relplot(data = metricas, kind = "line")

In [ ]:
# Salvando o modelo 
torch.save(modelo_dl.state_dict(), 'modelo_dl.pth')

In [ ]:
# Download do arquivo
from google.colab import files
files.download('modelo_dl.pth')

In [ ]:
# Para carregar o modelo salvo no Google Colab
modelo_final_1 = torch.load('modelo_dl.pth')

In [ ]:
print(modelo_final_1.keys())

In [ ]:
# Para carregar o modelo salvo em disco
files.upload()

In [ ]:
# Para carregar o modelo salvo no Google Colab
modelo_final_2 = torch.load('modelo_dl.pth')

In [ ]:
print(modelo_final_2.keys())

# Fim